In [2]:
!pip3 install telethon
!pip install nest_asyncio

You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
from telethon.sync import TelegramClient
from telethon import connection

import configparser
import json

# для корректного переноса времени сообщений в json
from datetime import date, datetime

# классы для работы с каналами
from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

# класс для работы с сообщениями
from telethon.tl.functions.messages import GetHistoryRequest

import nest_asyncio
nest_asyncio.apply()

In [4]:
!pip install nltk
import requests
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import pymorphy2
import pandas as pd
import json


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Считываем учетные данные
config = configparser.ConfigParser()
config.read("config.ini")

# Присваиваем значения внутренним переменным
api_id   = ''
api_hash = ''
username = ''

In [6]:
client = TelegramClient(username, api_id, api_hash)

In [6]:
client.start()

<coroutine object AuthMethods._start at 0x0000028AE58DB848>

In [7]:
async def dump_all_messages(channel, filename):
    """Записывает json-файл с информацией о всех сообщениях канала/чата"""
    offset_msg = 0    # номер записи, с которой начинается считывание
    limit_msg = 100   # максимальное число записей, передаваемых за один раз

    all_messages = []   # список всех сообщений
    total_messages = 0
    total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

    class DateTimeEncoder(json.JSONEncoder):
        '''Класс для сериализации записи дат в JSON'''
        def default(self, o):
            if isinstance(o, datetime):
                return o.isoformat()
            if isinstance(o, bytes):
                return list(o)
            return json.JSONEncoder.default(self, o)

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_msg = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break

    with open(filename, 'w', encoding='utf8') as outfile:
        json.dump(all_messages, outfile, ensure_ascii=False, cls=DateTimeEncoder)


In [8]:
async def main():
    url = 'https://t.me/doktorvladick'
    filename = 'doktorvladick.json'
    channel = await client.get_entity(url)
    await dump_all_messages(channel, filename)

async with client:
    client.loop.run_until_complete(main())

# Считываю teledict

In [9]:
import os
all_files = os.listdir('.../teledict')

all_teledict_post = []
for file in all_files:
    with open("teledict/"+file, 'r', encoding='utf8') as teledict_file:
        all_teledict_post.append(teledict_file.read())

In [10]:
len(all_teledict_post)

1281

In [11]:
all_teledict_post[0][0:100]

'Добрый вечер Андрей Артурович,так приятно видеть ваши видео и фотографии!\nНасыщенный график работы у'

# Считали посты в json, теперь анализ

In [95]:
f = open('.../teledict', encoding='utf8')
data = json.load(f)

In [122]:
public = 'teledict_analysis'
import os
os.makedirs(public, exist_ok=True)

In [12]:
# тольrо для teledict 
all_posts = all_teledict_post

In [97]:
all_posts = []
for i in range(len(data)):
    try:
        all_posts.append(data[i]['message'])
    except KeyError:
        pass
##for i in range(len(data2)):
    #try:
        #all_posts.append(data2[i]['message'])
    #except KeyError:
        #pass

In [124]:
len(all_posts)

1281

In [13]:
all_posts_full = all_posts.copy()
all_posts_for_tonalnost = all_posts.copy()

In [14]:
all_posts = ''.join(all_posts)
all_posts = all_posts.lower()
spec_chars = string.punctuation + '\n\xa0«»\t—…' 

MemoryError: 

In [127]:
all_posts = "".join([ch for ch in all_posts if ch not in spec_chars])

MemoryError: 

In [ ]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return ' '.join(res)

In [103]:
all_posts = lemmatize(all_posts)

In [104]:
text_tokens = word_tokenize(all_posts)
text = nltk.Text(text_tokens)

In [105]:
# расширить
stop_ru = pd.read_csv('.../stop_ru.txt')
russian_stopwords.extend(['это', 'нею', '-q', ])
russian_stopwords.extend(stop_ru['и'].values)
filtered_text = [word for word in text if word not in russian_stopwords]
fdist = FreqDist(filtered_text)

In [106]:
with open(public+'/Частотные_слова_c_леммитизацией_wrongfem.txt', 'w+') as the_file:
    for w in fdist.most_common(1000):
        the_file.write(str(w)+'\n')
        print(w)

('женщина', 1436)
('мужчина', 1100)
('ребёнок', 758)
('–', 473)
('мужик', 352)
('год', 345)
('говорить', 319)
('работа', 314)
('девушка', 285)
('муж', 274)
('должный', 266)
('делать', 264)
('хороший', 257)
('жена', 252)
('семья', 248)
('знать', 241)
('дом', 223)
('работать', 218)
('дело', 212)
('стать', 209)
('мужской', 207)
('вообще', 204)
('женский', 195)
('жить', 185)
('большой', 183)
('отношение', 178)
('деньга', 168)
('нужный', 162)
('сделать', 162)
('самый', 159)
('девочка', 154)
('мама', 154)
('баба', 152)
('право', 151)
('думать', 146)
('считать', 146)
('никто', 141)
('история', 140)
('идти', 138)
('понимать', 136)
('чтоть', 135)
('развод', 131)
('любой', 123)
('поэтому', 118)
('мать', 117)
('отец', 117)
('канал', 116)
('вопрос', 113)
('понять', 113)
('квартира', 113)
('смотреть', 112)
('давать', 110)
('видеть', 110)
('слово', 108)
('писать', 108)
('любить', 107)
('насилие', 106)
('сразу', 105)
('сила', 105)
('проблема', 105)
('рука', 104)
('домашний', 103)
('собственный', 100)

# Биграммы

In [107]:
bgs = nltk.bigrams(filtered_text)
fdist_bigs = nltk.FreqDist(bgs)
with open(public+'/Биграммы_с_леммой_wrongfem.txt', 'w+') as the_file:
    for w in fdist_bigs.most_common(10000):
        the_file.write(str(w)+'\n')
        print(w)

(('женщина', 'должный'), 41)
(('мужчина', 'женщина'), 40)
(('домашний', 'насилие'), 32)
(('женщина', 'мужчина'), 29)
(('женщина', 'ребёнок'), 26)
(('точка', 'зрение'), 20)
(('друг', 'друг'), 19)
(('выйти', 'замуж'), 19)
(('говорить', 'женщина'), 17)
(('домашний', 'дело'), 16)
(('право', 'женщина'), 16)
(('обращать', 'внимание'), 16)
(('собственный', 'ребёнок'), 16)
(('двойной', 'стандарт'), 16)
(('чтоть', 'делать'), 15)
(('мужчина', 'мужчина'), 14)
(('рожать', 'ребёнок'), 14)
(('уход', 'ребёнок'), 14)
(('бывший', 'муж'), 14)
(('сильный', 'пол'), 14)
(('мужчина', 'сильный'), 13)
(('дом', 'сидеть'), 13)
(('никто', 'нужный'), 13)
(('нормальный', 'мужик'), 13)
(('женщина', 'женщина'), 13)
(('дом', 'ребёнок'), 13)
(('мужчина', '–'), 13)
(('мама', 'мама'), 13)
(('ребёнок', '–'), 13)
(('сильный', 'женщина'), 12)
(('слабый', 'женщина'), 12)
(('растить', 'ребёнок'), 12)
(('зарабатывать', 'большой'), 12)
(('конец', 'конец'), 11)
(('10', 'год'), 11)
(('сделать', 'аборт'), 11)
(('сидеть', 'дом'), 

UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4a3' in position 15: character maps to <undefined>

# Биграммы без лемитизации

In [108]:
all_posts_full = ''.join(all_posts_full)
all_posts_full = all_posts_full.lower()
all_posts_full = "".join([ch for ch in all_posts_full if ch not in spec_chars])

In [109]:
text_tokens = word_tokenize(all_posts_full)
text = nltk.Text(text_tokens)

stop_ru = pd.read_csv('.../stop_ru.txt')
russian_stopwords.extend(['это', 'нею', '-q', ])
russian_stopwords.extend(stop_ru['и'].values)
filtered_text_full = [word for word in text if word not in russian_stopwords]

In [110]:
bgs = nltk.bigrams(filtered_text_full)
fdist_bigs = nltk.FreqDist(bgs)
with open(public+'/Биграммы_без_леммы_wrongfem.txt', 'w+') as the_file:
    for w in fdist_bigs.most_common(10000):
        the_file.write(str(w)+'\n')
        print(w)

(('женщина', 'должна'), 22)
(('домашнего', 'насилия'), 19)
(('точки', 'зрения'), 16)
(('первую', 'очередь'), 16)
(('друг', 'друга'), 14)
(('чтото', 'делать'), 14)
(('сильный', 'пол'), 14)
(('женщины', 'должны'), 13)
(('имеет', 'право'), 13)
(('рожать', 'детей'), 12)
(('конце', 'концов'), 11)
(('никому', 'нужна'), 10)
(('семейной', 'жизни'), 10)
(('растить', 'детей'), 10)
(('вышел', 'новый'), 9)
(('истории', 'подписчиц'), 9)
(('логика', 'говорить'), 9)
(('портить', 'отношения'), 9)
(('двое', 'детей'), 9)
(('огромное', 'количество'), 8)
(('instagram', 'вышел'), 8)
(('бокал', 'вина'), 8)
(('мужчин', 'женщин'), 8)
(('женщина', 'хочет'), 8)
(('бывший', 'муж'), 8)
(('готовить', 'еду'), 8)
(('площадкиvk', 'httpsvkcomwrongfeminstagram'), 8)
(('двойных', 'стандартов'), 8)
(('лучшем', 'случае'), 7)
(('нормальный', 'мужик'), 7)
(('прав', 'женщин'), 7)
(('фем', 'повестки'), 7)
(('женщин', 'женщин'), 7)
(('новый', 'пост'), 7)
(('заводить', 'детей'), 7)
(('коем', 'случае'), 7)
(('среди', 'ночи'), 7)

UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4a3' in position 16: character maps to <undefined>

# Тональность текста

In [29]:
!pip install dostoevsky
!python -m dostoevsky download fasttext-social-network-model
import dostoevsky
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

  Using cached dostoevsky-0.6.0-py2.py3-none-any.whl (8.5 kB)


    ERROR: Command errored out with exit status 1:
     command: 'c:\python37\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\eduar\\AppData\\Local\\Temp\\pip-install-tk3fgmih\\fasttext\\setup.py'"'"'; __file__='"'"'C:\\Users\\eduar\\AppData\\Local\\Temp\\pip-install-tk3fgmih\\fasttext\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\eduar\AppData\Local\Temp\pip-record-f3aow15a\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\python37\Include\fasttext'
         cwd: C:\Users\eduar\AppData\Local\Temp\pip-install-tk3fgmih\fasttext\
    Complete output (18 lines):
    running install
    running build
    running build_py
    creating build
    creating build\lib.win-amd64-3.7
    creating build\lib.win-amd64-3.7\fasttext
    copying python\fasttext_module\fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
Using legacy setup.py install for fasttext, since package 'wheel' is not installed.
    Running setup.py install for fasttext: started
    Running setup.py install for fasttext: finished with status 'error'


C:\Python37\python.exe: No module named dostoevsky


ModuleNotFoundError: No module named 'dostoevsky'

In [ ]:
messages = all_posts_for_tonalnost
results = model.predict(messages, k=2)
count_neg = 0
count_pos = 0
count_neutral = 0
neg_list = []
for message, sentiment in zip(messages, results):
    if 'neutral' in sentiment.keys():
        if sentiment['neutral'] > 0.1:
            count_neutral = count_neutral + 1
    if 'negative' in sentiment.keys():
        if sentiment['negative'] > 0.1:
            count_neg = count_neg + 1
            neg_list.append([message, sentiment['negative']])
    if 'positive' in sentiment.keys():
        if sentiment['positive'] > 0.1:
            count_pos = count_pos + 1

In [ ]:
# процент нейтральных постов
count_neutral*100/len(all_posts_for_tonalnost)

In [ ]:
# процент позитивных постов
count_pos*100/len(all_posts_for_tonalnost)

In [ ]:
# процент негативных постов
count_neg*100/len(all_posts_for_tonalnost)

# N-граммы

In [111]:
from nltk.util import ngrams

### C лемитизацией

In [112]:
ngs = nltk.ngrams(filtered_text, n = 3)
fdist_nigs = nltk.FreqDist(ngs)
with open(public+'/3граммы_с_леммой_wrongfem.txt', 'w+') as the_file:
    for w in fdist_nigs.most_common(10000):
        the_file.write(str(w)+'\n')
        print(w)

(('мужчина', 'сильный', 'пол'), 11)
(('instagram', 'выйти', 'новый'), 8)
(('история', 'подписчица', 'ламповый'), 7)
(('защита', 'право', 'женщина'), 7)
(('выйти', 'новый', 'пост'), 7)
(('пить', 'избивать', 'жена'), 7)
(('fem', 'revolutionканал', 'помогать'), 6)
(('revolutionканал', 'помогать', 'женщина'), 6)
(('помогать', 'женщина', 'режим'), 6)
(('женщина', 'режим', 'реальный'), 6)


UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4a3' in position 24: character maps to <undefined>

In [89]:
ngs = nltk.ngrams(filtered_text, n = 4)
fdist_nigs = nltk.FreqDist(ngs)
for w in fdist_nigs.most_common(10000):
    print(w)

(('поднимать', 'женский', 'самооценка', 'дно'), 16)
(('женский', 'самооценка', 'дно', 'марианский'), 14)
(('самооценка', 'дно', 'марианский', 'впадина'), 13)
(('публиковать', 'англоязычный', 'контент', 'авторский'), 12)
(('англоязычный', 'контент', 'авторский', 'перевод'), 12)
(('контент', 'авторский', 'перевод', 'рассказывать'), 12)
(('авторский', 'перевод', 'рассказывать', 'феминизм'), 12)
(('перевод', 'рассказывать', 'феминизм', 'сша'), 12)
(('рассказывать', 'феминизм', 'сша', 'русскоговорящий'), 12)
(('феминизм', 'сша', 'русскоговорящий', 'женщина'), 12)
(('сша', 'русскоговорящий', 'женщина', 'нужный'), 12)
(('русскоговорящий', 'женщина', 'нужный', 'поддержка'), 12)
(('женщина', 'нужный', 'поддержка', 'осознание'), 12)
(('нужный', 'поддержка', 'осознание', 'своей'), 12)
(('поддержка', 'осознание', 'своей', 'сила'), 12)
(('брать', 'интервью', 'простой', 'американец'), 12)
(('интервью', 'простой', 'американец', 'разный'), 12)
(('простой', 'американец', 'разный', 'актуальный'), 12)


(('удалённый', 'группа', 'выписать', 'предупреждение'), 1)
(('группа', 'выписать', 'предупреждение', 'враждебный'), 1)
(('выписать', 'предупреждение', 'враждебный', 'действие'), 1)
(('предупреждение', 'враждебный', 'действие', 'ждать'), 1)
(('враждебный', 'действие', 'ждать', 'ответ'), 1)
(('действие', 'ждать', 'ответ', 'поддержка'), 1)
(('ждать', 'ответ', 'поддержка', '🤡писать'), 1)
(('ответ', 'поддержка', '🤡писать', 'отзыв'), 1)
(('поддержка', '🤡писать', 'отзыв', 'гугл'), 1)
(('🤡писать', 'отзыв', 'гугл', 'плей'), 1)
(('отзыв', 'гугл', 'плей', 'appstore'), 1)
(('гугл', 'плей', 'appstore', 'администрация'), 1)
(('плей', 'appstore', 'администрация', 'вк'), 1)
(('appstore', 'администрация', 'вк', 'относиться'), 1)
(('администрация', 'вк', 'относиться', 'девушкамhttpsvkcommobile25'), 1)
(('вк', 'относиться', 'девушкамhttpsvkcommobile25', 'декабрь'), 1)
(('относиться', 'девушкамhttpsvkcommobile25', 'декабрь', '1800'), 1)
(('девушкамhttpsvkcommobile25', 'декабрь', '1800', 'случиться'), 1)
(

(('некоторый', 'группа', 'население', 'десятилетие'), 1)
(('группа', 'население', 'десятилетие', 'xxi'), 1)
(('население', 'десятилетие', 'xxi', 'век'), 1)
(('десятилетие', 'xxi', 'век', 'случай'), 1)
(('xxi', 'век', 'случай', 'фиксировать'), 1)
(('век', 'случай', 'фиксировать', 'континент'), 1)
(('случай', 'фиксировать', 'континент', 'glasnayamedia'), 1)
(('фиксировать', 'континент', 'glasnayamedia', 'рассказывать'), 1)
(('континент', 'glasnayamedia', 'рассказывать', 'женщина'), 1)
(('glasnayamedia', 'рассказывать', 'женщина', 'сей'), 1)
(('рассказывать', 'женщина', 'сей', 'стерилизовать'), 1)
(('женщина', 'сей', 'стерилизовать', 'насильственно'), 1)
(('сей', 'стерилизовать', 'насильственно', 'принудительно'), 1)
(('стерилизовать', 'насильственно', 'принудительно', 'откуда'), 1)
(('насильственно', 'принудительно', 'откуда', 'пойти'), 1)
(('принудительно', 'откуда', 'пойти', 'практика'), 1)
(('откуда', 'пойти', 'практика', '👉'), 1)
(('пойти', 'практика', '👉', 'читать'), 1)
(('практика'

(('цитата', 'пётр', '1', '🤣httpsvkcomwall203991181143207thread143209ряяяяять'), 1)
(('пётр', '1', '🤣httpsvkcomwall203991181143207thread143209ряяяяять', 'купить'), 1)
(('1', '🤣httpsvkcomwall203991181143207thread143209ряяяяять', 'купить', 'растворимый'), 1)
(('🤣httpsvkcomwall203991181143207thread143209ряяяяять', 'купить', 'растворимый', 'кофе'), 1)
(('купить', 'растворимый', 'кофе', 'должный'), 1)
(('растворимый', 'кофе', 'должный', 'дать'), 1)
(('кофе', 'должный', 'дать', 'вернуть'), 1)
(('должный', 'дать', 'вернуть', 'деньги😭😭😭😭вообщеть'), 1)
(('дать', 'вернуть', 'деньги😭😭😭😭вообщеть', 'заплатить'), 1)
(('вернуть', 'деньги😭😭😭😭вообщеть', 'заплатить', 'кофе'), 1)
(('деньги😭😭😭😭вообщеть', 'заплатить', 'кофе', 'обязать'), 1)
(('заплатить', 'кофе', 'обязать', 'выслушивать'), 1)
(('кофе', 'обязать', 'выслушивать', 'истерика'), 1)
(('обязать', 'выслушивать', 'истерика', 'мужчинкиряяяяяяяяять'), 1)
(('выслушивать', 'истерика', 'мужчинкиряяяяяяяяять', 'баба'), 1)
(('истерика', 'мужчинкиряяяяяяяяя

(('большой', 'деньга', 'прилететь', 'карма'), 1)
(('деньга', 'прилететь', 'карма', 'ветеран'), 1)
(('прилететь', 'карма', 'ветеран', 'россия'), 1)
(('карма', 'ветеран', 'россия', 'потребовать'), 1)
(('ветеран', 'россия', 'потребовать', 'закрыть'), 1)
(('россия', 'потребовать', 'закрыть', 'канал'), 1)
(('потребовать', 'закрыть', 'канал', 'антиконституционный'), 1)
(('закрыть', 'канал', 'антиконституционный', 'риторикучтый'), 1)
(('канал', 'антиконституционный', 'риторикучтый', 'думаетеонить'), 1)
(('антиконституционный', 'риторикучтый', 'думаетеонить', 'готовить'), 1)
(('риторикучтый', 'думаетеонить', 'готовить', 'могутести'), 1)
(('думаетеонить', 'готовить', 'могутести', 'искать'), 1)
(('готовить', 'могутести', 'искать', 'стажировка'), 1)
(('могутести', 'искать', 'стажировка', 'грант'), 1)
(('искать', 'стажировка', 'грант', 'мероприятие'), 1)
(('стажировка', 'грант', 'мероприятие', 'посвятить'), 1)
(('грант', 'мероприятие', 'посвятить', 'женщина'), 1)
(('мероприятие', 'посвятить', 'жен

(('подобный', 'грязь', 'сторона', 'женщина'), 1)
(('грязь', 'сторона', 'женщина', 'минимум'), 1)
(('сторона', 'женщина', 'минимум', 'индифферентный'), 1)
(('женщина', 'минимум', 'индифферентный', 'выгодный'), 1)
(('минимум', 'индифферентный', 'выгодный', 'знать'), 1)
(('индифферентный', 'выгодный', 'знать', 'женщина'), 1)
(('выгодный', 'знать', 'женщина', 'пытаться'), 1)
(('знать', 'женщина', 'пытаться', 'договориться'), 1)
(('женщина', 'пытаться', 'договориться', 'похороший'), 1)
(('пытаться', 'договориться', 'похороший', 'говорить'), 1)
(('договориться', 'похороший', 'говорить', 'мирный'), 1)
(('похороший', 'говорить', 'мирный', 'диалог'), 1)
(('говорить', 'мирный', 'диалог', 'хуятина'), 1)
(('мирный', 'диалог', 'хуятина', 'невозможный'), 1)
(('диалог', 'хуятина', 'невозможный', 'замечать'), 1)
(('хуятина', 'невозможный', 'замечать', 'настолько'), 1)
(('невозможный', 'замечать', 'настолько', 'поебать'), 1)
(('замечать', 'настолько', 'поебать', 'мнение'), 1)
(('настолько', 'поебать', 

### Без лемитизации

In [113]:
ngs = nltk.ngrams(filtered_text_full, n = 3)
fdist_nigs = nltk.FreqDist(ngs)
with open(public+'/3граммы_без_леммы_wrongfem.txt', 'w+') as the_file:
    for w in fdist_nigs.most_common(10000):
        the_file.write(str(w)+'\n')
        print(w)

(('instagram', 'вышел', 'новый'), 8)
(('вышел', 'новый', 'пост'), 7)
(('мужчина', 'сильный', 'пол'), 7)
(('fem', 'revolutionканал', 'помогает'), 6)
(('revolutionканал', 'помогает', 'женщинам'), 6)
(('помогает', 'женщинам', 'режиме'), 6)
(('женщинам', 'режиме', 'реального'), 6)


UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f4a3' in position 26: character maps to <undefined>

### Частотные слова перед словами мужчина и женщина

In [114]:
# лемитизация мужчина
text_to_find_words = ' '.join(filtered_text)
mans_words = []
for tags in text_to_find_words.split('мужчина'):
    try:
        mans_words.append(tags.split()[-1])
    except:
        continue
        
fdist = FreqDist(mans_words)
with open(public+'/ЧС_с_леммой_мужчина_wrongfem.txt', 'w+') as the_file:
    for w in fdist.most_common(100):
        the_file.write(str(w)+'\n')
        print(w)

('женщина', 31)
('видеть', 9)
('большинство', 8)
('знать', 8)
('многий', 8)
('сторона', 7)
('ненавидеть', 7)
('российский', 7)
('нужный', 7)
('отношение', 6)
('современный', 6)
('плохой', 6)
('разный', 6)
('год', 6)
('дорогой', 6)
('право', 6)
('понимать', 5)
('рука', 5)
('писать', 5)
('нравиться', 5)
('жить', 5)
('существовать', 5)
('многие', 5)
('“', 4)
('маленький', 4)
('ущемление', 4)
('избежать', 4)
('зарабатывать', 4)
('молчать', 4)
('число', 4)
('зрение', 4)
('любой', 4)
('считать', 4)
('найти', 4)
('среднестатистический', 4)
('обязательство', 4)
('тварь', 4)
('взрослый', 4)
('убивать', 4)
('ребёнок', 4)
('ненависть', 4)
('оплата', 4)
('сильный', 4)
('нормальный', 4)
('женщин', 4)
('дом', 3)
('смеяться', 3)
('помощь', 3)
('большой', 3)
('смерть', 3)
('–', 3)
('прямо', 3)
('ведический', 3)
('представление', 3)
('семья', 3)
('написать', 3)
('супруг', 3)
('новый', 3)
('понять', 3)
('насколько', 3)
('разом', 3)
('добыча', 3)
('заинтересовать', 3)
('фраза', 3)
('русский', 3)
('работа

In [115]:
# без лемитизации мужчина
text_to_find_words = ' '.join(filtered_text_full)
mans_words = []
for tags in text_to_find_words.split('мужчина'):
    try:
        mans_words.append(tags.split()[-1])
    except:
        continue
fdist = FreqDist(mans_words)
with open(public+'/ЧС_без_леммы_мужчина_wrongfem.txt', 'w+') as the_file:
    for w in fdist.most_common(100):
        the_file.write(str(w)+'\n')
        print(w)

('женщина', 6)
('руку', 4)
('женщин', 4)
('женщины', 4)
('сильный', 4)
('нужен', 4)
('зарабатывает', 3)
('добыча', 3)
('корне', 3)
('мужчиныдело', 3)
('подразумевается', 3)
('фу', 3)
('малознакомым', 3)
('знаю', 3)
('добивается', 3)
('шлюха', 3)
('повезло', 2)
('–', 2)
('женщинах', 2)
('доводам', 2)
('уйти', 2)
('немногочисленными', 2)
('российский', 2)
('нравиться', 2)
('нужнее', 2)
('поняла', 2)
('ребенку', 2)
('эгомомент', 2)
('спросить', 2)
('уберегла', 2)
('такая', 2)
('отбился', 2)
('подтирать', 2)
('мечтааааа', 2)
('любой', 2)
('относитесь', 2)
('мире', 2)
('пушку', 2)
('наравне', 2)
('появился', 2)
('мусульманам', 2)
('жить', 2)
('ненависти', 2)
('предложение', 2)
('настоящий', 2)
('целый', 2)
('сложного', 2)
('разгружать', 2)
('другим', 2)
('вообще', 2)
('нуждаются', 2)
('манипулирующая', 2)
('отношения', 2)
('направлять', 2)
('несомненно', 1)
('любят', 1)
('дома', 1)
('поддержкой', 1)
('изнасилования', 1)
('ибо', 1)
('стороны', 1)
('подводилосначала', 1)
('связываются', 1)
('

#### женщина

In [116]:
# лемитизация
text_to_find_words = ' '.join(filtered_text)
mans_words = []
for tags in text_to_find_words.split('женщина'):
    try:
        mans_words.append(tags.split()[-1])
    except:
        continue
        
fdist = FreqDist(mans_words)
for w in fdist.most_common(100):
    print(w)
with open(public+'/ЧС_с_леммой_женщина_wrongfem.txt', 'w+') as the_file:
    for w in fdist.most_common(100):
        the_file.write(str(w)+'\n')
        print(w)

('мужчина', 41)
('говорить', 17)
('право', 16)
('слабый', 15)
('сильный', 12)
('дело', 12)
('помогать', 10)
('настоящий', 10)
('видеть', 9)
('считать', 9)
('делать', 9)
('отношение', 9)
('общество', 8)
('работать', 8)
('семья', 8)
('многий', 7)
('работа', 7)
('тело', 7)
('многие', 7)
('большинство', 7)
('придумать', 7)
('ребёнок', 6)
('мол', 6)
('обычный', 6)
('понимать', 6)
('развести', 6)
('маленький', 6)
('предназначение', 5)
('–', 5)
('правило', 5)
('дамыканал', 5)
('поносить', 5)
('любимый', 5)
('порядочный', 5)
('война', 5)
('случай', 5)
('угодный', 5)
('рука', 5)
('ненависть', 5)
('деньга', 5)
('искать', 4)
('бывать', 4)
('жить', 4)
('канал', 4)
('рассказывать', 4)
('красивый', 4)
('хороший', 4)
('слышать', 4)
('большой', 4)
('история', 4)
('предлагать', 4)
('нормальный', 4)
('проблема', 4)
('мужик', 4)
('сделать', 4)
('значимость', 4)
('одинокий', 4)
('должный', 4)
('душа', 4)
('проходить', 4)
('туча', 4)
('материнство', 4)
('природа', 4)
('создать', 4)
('насилие', 4)
('подряд'

In [117]:
# без лемитизации
text_to_find_words = ' '.join(filtered_text_full)
mans_words = []
for tags in text_to_find_words.split('женщина'):
    try:
        mans_words.append(tags.split()[-1])
    except:
        continue

fdist = FreqDist(mans_words)
with open(public+'/ЧС_без_леммы_женщина_wrongfem.txt', 'w+') as the_file:
    for w in fdist.most_common(100):
        the_file.write(str(w)+'\n')
        print(w)

('помогает', 7)
('сильная', 6)
('слабая', 6)
('обычная', 5)
('другими', 4)
('работу', 4)
('настоящая', 4)
('мужчин', 4)
('мужчины', 4)
('материнстве', 4)
('оказывается', 3)
('многим', 3)
('порядочная', 3)
('случаи', 3)
('нелогично', 3)
('нормальнаято', 3)
('говорить', 3)
('старая', 3)
('обращаться', 3)
('слышим', 3)
('мешает', 3)
('осторожна', 3)
('другим', 3)
('одна', 3)
('мужчинам', 2)
('слабых', 2)
('взяться', 2)
('слабой', 2)
('–', 2)
('пост', 2)
('самая', 2)
('дело', 2)
('голос', 2)
('мужика', 2)
('момент', 2)
('красивая', 2)
('поэтому', 2)
('мол', 2)
('объяснить', 2)
('годами', 2)
('семьях', 2)
('поскольку', 2)
('многочисленными', 2)
('хорошая', 2)
('пропаганда', 2)
('выступает', 2)
('мужчинами', 2)
('натворила', 2)
('угодно', 2)
('работающая', 2)
('отношения', 2)
('неизвестная', 2)
('гдето', 2)
('запоёшь', 2)
('горбачусь', 2)
('дойти', 2)
('сказка', 2)
('гнёздышко', 2)
('масле', 2)
('творцом', 2)
('покорятся', 2)
('зубам', 2)
('науку', 2)
('ненависти', 2)
('природы', 2)
('нормал

In [ ]:
all_posts[1:1000]

In [40]:
import re
def searchWordinSentence(word,sentence):
    pattern = re.compile(' '+word+' |^'+word+' | '+word+' $')
    if re.search(pattern,sentence):
        return True

In [45]:
all_sentences = ' '.join(all_posts_for_tonalnost)
word = "тварь"
all_sentences = [ t for t in all_sentences.split('. ') if searchWordinSentence(word,t)]
for sent in all_sentences:
    print('========================================')
    print(sent)

Пошёл тварь блядь 🤣👍

В Хабаровской школе учитель физики (бывший военный) откуколдил омежку старшеклассника
